Code sourced from: <br>
https://www.kaggle.com/code/stpeteishii/dice-yolov8-train-and-predict-2#Create-yaml-file
<br>

used solely for testing purposes!

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nellbyler/d6-dice")

ANNOTATION_PATH = path + "\\d6-dice\\Annotations"
IMAGES_PATH = path + "\\d6-dice\\Images"

print("Path to dataset files:", path)

C:\Users\samal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\samal\.cache\kagglehub\datasets\nellbyler\d6-dice\versions\3


In [2]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.39  Python-3.11.9 torch-2.5.1+cpu CPU (AMD Ryzen 7 5800X 8-Core Processor)
Setup complete  (16 CPUs, 31.9 GB RAM, 813.5/930.6 GB disk)


In [3]:
train_path = 'datasets/train/'
valid_path = 'datasets/valid/'
test_path = 'datasets/test/'

In [4]:
# print(os.path.join(path, '/kaggle/input/d6-dice/d6-dice/Annotations'))


ano_paths=[]
for dirname, _, filenames in os.walk(ANNOTATION_PATH):
    for filename in filenames:
        if filename[-4:]=='.txt':
            ano_paths+=[(os.path.join(dirname, filename))]

ano_paths.pop(0);

n=len(ano_paths) 
print(n)
N=list(range(n))
random.shuffle(N)

train_ratio = 0.6
valid_ratio = 0.2
test_ratio = 0.2

train_size = int(train_ratio*n)
valid_size = int(valid_ratio*n)

train_i = N[:train_size]
valid_i = N[train_size:train_size+valid_size]
test_i = N[train_size+valid_size:]

# print(len(train_i))
# print(len(valid_i))
# print(len(test_i))

250


In [5]:
if not os.path.exists('datasets/train/images'):
    os.makedirs('datasets/train/images')
if not os.path.exists('datasets/valid/images'):
    os.makedirs('datasets/valid/images')
if not os.path.exists('datasets/test/images'):
    os.makedirs('datasets/test/images')
if not os.path.exists('datasets/train/labels'):
    os.makedirs('datasets/train/labels')
if not os.path.exists('datasets/valid/labels'):
    os.makedirs('datasets/valid/labels')
if not os.path.exists('datasets/test/labels'):
    os.makedirs('datasets/test/labels')

delim = '\\'

for i in train_i:
    ano_path=ano_paths[i]
    img_path=os.path.join(IMAGES_PATH,
                          ano_path.split(delim)[-1][0:-4]+'.jpg')
    
    shutil.copy(ano_path, os.path.join(train_path + "\\labels\\", os.path.basename(ano_path)))
    shutil.copy(img_path, os.path.join(train_path + "\\images\\", os.path.basename(img_path)))

for i in valid_i:
    ano_path=ano_paths[i]
    img_path=os.path.join(IMAGES_PATH,
                          ano_path.split(delim)[-1][0:-4]+'.jpg')
    
    shutil.copy(ano_path, os.path.join(valid_path + "\\labels\\", os.path.basename(ano_path)))
    shutil.copy(img_path, os.path.join(valid_path + "\\images\\", os.path.basename(img_path)))

for i in test_i:
    ano_path=ano_paths[i]
    img_path=os.path.join(IMAGES_PATH,
                          ano_path.split(delim)[-1][0:-4]+'.jpg')
    
    shutil.copy(ano_path, os.path.join(test_path + "\\labels\\", os.path.basename(ano_path)))
    shutil.copy(img_path, os.path.join(test_path + "\\images\\", os.path.basename(img_path)))


# def count_files_in_subdirectories(path):
#     total_files = 0
#     for root, dirs, files in os.walk(path):
#         total_files += len(files)
#     return total_files

# print("Number of files in train directory:", count_files_in_subdirectories(train_path))
# print("Number of files in valid directory:", count_files_in_subdirectories(valid_path))
# print("Number of files in test directory:", count_files_in_subdirectories(test_path))

In [10]:
import yaml

source_path = 'C:\\Users\\samal\\Computer-vision-project\\DiceRollDetection'
#source_path = 'C:\\Users\\leeal\\OneDrive - Virginia Tech\\School\\Computer Vision\\DiceRollDetection'

train_path = source_path+'\\datasets\\train'
test_path = source_path+'\\datasets\\test'
valid_path = source_path+'\\datasets\\valid'

data_yaml = dict(
    train =train_path,
    val =valid_path,
    test=test_path,
    nc =6,
    names =list('123456')
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [11]:
names =list('123456')
M=list(range(len(names)))
class_map=dict(zip(M,names))

In [12]:
model = YOLO("yolov8x.pt") 

In [ ]:
!yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=12 imgsz=480